# Reto 2: Problema multiclase

Este reto consiste en aprender a clasificar 4 tipos diferentes de vehículos utilizando cualquiera de los clasificadores o técnicas estudiadas hasta el momento. Esto incluye:
+ clasificación lineal
+ transformaciones no lineales seguido de un clasificador lineal
+ Support Vector Machines (SVM)
+ Decision Tree (DT)

Además se pueden aplicar técnicas de preprocesado como:
+ escalado de las características
+ *grid search* para búsqueda de hiperparámetros
+ validación cruzada

El conjunto de datos, *vehiculos_reto2.csv*, consiste en 592 muestras de vehículos; cada uno de ellos representado por 18 características.

Para evaluar las propuestas se utilizará un conjunto de datos que se mantendrá oculto hasta después de la entrega

### Requisitos
+ La entrega se realiza **sólo** a través de la tarea habilitada para ello en la pestaña de *Evaluación* del Aula Virtual.
+ Se debe entregar un cuaderno Jupyter con el nombre de los participantes.<br>
  *Por ejemplo*:   **Cuesta_LeCunn.ipynb**
+ El cuaderno entregado debe seguir la estructura y reglas de este cuaderno

### Competición
+ Todos los cuadernos entregados se subirán al repo de GitHub y se ejecutarán en Binder, donde ya estará en conjunto de test que permanecía oculto.
+ El número de aciertos respecto del número de ejemplos será la puntuación del reto.
+ **Importante** Es muy fácil asegurarte de que tu código funcionará bien. Para ello:
    1. Agrupa todo tu código en una única celda
    2. En el cuaderno del reto que hay en Binder: elimina las celdas que hay entre la verde y la roja, y copia tu celda entre ellas.
    3. Ejecuta ese cuaderno de Binder. 
    
### Plazo: lunes 26 de oct. de 2020 a las 6 am.
Es decir, incluye toda la noche del domingo 25 de oct.


---
    [ES] Código de Alfredo Cuesta Infante para 'Reconocimiento de Patrones'
       @ Master Universitario en Visión Artificial, 2020, URJC (España)
    [EN] Code by Alfredo Cuesta-Infante for 'Pattern Recognition'
       @ Master of Computer Vision, 2020, URJC (Spain)

    alfredo.cuesta@urjc.es

In [1]:
# Conjunto distribuido para el reto

Challange_filename = '../../Datasets/vehiculos_reto2.csv'

In [2]:
# Conjunto NO distribuido para evaluar los clasificadores entregados

Test_filename = '../../Datasets/vehiculos_test.csv' #<-- este nombre cambiará después del plazo de entrega

In [3]:
#-[1]. Load data from CSV and put all in a single dataframe 'FullSet'

import numpy  as np
import pandas as pd
from matplotlib import pyplot as plt
import sys
sys.path.append('../../MyUtils/')
import MyUtils as my
seed = 1234 #<- random generator seed (comment to get randomness)

#-[2]. Load data from CSV and put all in a single dataframe 'FullSet'

FullSet = pd.read_csv(Challange_filename, header=0)
FullX = FullSet.drop('Class', axis=1)
FullY = FullSet[['Class']]

<table style="width:100%;"> 
 <tr style='background:lime'>
  <td style="text-align:left">
      <h2>Tu código debe empezar a partir de aquí y puede tener tantas celdas como quieras</h2>
      <p> Si quieres, puedes borrar (o convertir en RawNBConvert) las celdas de ejemplo
      <h3>Importante:</h3>
      <p>Tu código debe producir las siguientes variables: </p>
      <p> $\quad \bullet$ <b>clf:</b> el clasificador final con el que se realizará el test<br>
       $\quad \bullet$ <b>X_test:</b> el conjunto de test listo para ser usado por el método <b>predict</b><br>
       $\quad \bullet$ <b>Y_test:</b> es el vector de etiquetas del conjunto de X_test listo para ser usado por el método <b>confusion_matrix</b>
      </p>
  </td>
 </tr>
</table>

In [4]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

nombres = ["Jose Miguel Zamora Bautista", "Antonio Triguero Noriega"]

scaler = MinMaxScaler()
FullX = scaler.fit_transform(FullX)
onehot = OneHotEncoder()
onehot.fit(FullY)
FullY = onehot.transform(FullY).argmax(axis=1)
FullY = np.array([x[0] for x in FullY.tolist()])

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(FullX, FullY, test_size=0.1, random_state=42)

from sklearn.svm import SVC, LinearSVC
from sklearn import tree

clf = SVC(kernel='rbf', random_state = seed)

from sklearn.model_selection import GridSearchCV
import numpy as np

max_degree = 15
max_C = 2.5

svc_grid_params = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': list(range(1, max_degree)),
    'gamma': ['scale', 'auto'],
    'C': list(np.arange(0, max_C, 0.1))
}

grid = GridSearchCV(clf, svc_grid_params, n_jobs=-1)
grid.fit(x_train, y_train)

clf = grid.best_estimator_

X_test = x_test
Y_test = y_test

'''RESULTADO: X_test es el dataframe para utilizar en >>> Y_pred = clf.predict() 
   RESULTADO: Y_test es el array con las etiquetas para utilizar en >>> confusion_matrix(Y_test,Y_pred)
'''

'RESULTADO: X_test es el dataframe para utilizar en >>> Y_pred = clf.predict() \n   RESULTADO: Y_test es el array con las etiquetas para utilizar en >>> confusion_matrix(Y_test,Y_pred)\n'

<table style="width:100%;"> 
 <tr style='background:pink'>
  <td style="text-align:left">
      <h2>A partir de aquí ya no se pueden modificar las celdas</h2>
          <h3>Comprueba que:</h3>
          <p> $\quad \bullet$ tu clasificador está almacenado en la variable <b>clf</b><br>
              $\quad \bullet$ tienes el conjunto de test correctamente almacenado en la variable <b>X_test</b><br>
              $\quad \bullet$ tienes las etiquetas del conjunto de test correctamente almacenadas en la variable <b>Y_test</b><br>
          </p>
      
  </td>
 </tr>
</table>

## Test

In [5]:
from sklearn.metrics import confusion_matrix

Y_hat = clf.predict(X_test)
conf_mat = confusion_matrix(Y_test , Y_hat)
N_success  = np.trace(conf_mat)
N_fails = Y_test.shape[0]-N_success
#-------------------------------
print (nombres,"\n")
print("Confusion matrix:\n")
print(conf_mat,"\n")
print("Outcome:\n")
strlog = "  :) HIT  = %d, (%0.2f%%)"%(N_success, 100*N_success/(N_success+N_fails))
print(strlog)
strlog = "  :( FAIL = %d, (%0.2f%%)"%(N_fails, 100*N_fails/(N_success+N_fails))
print(strlog)

['Jose Miguel Zamora Bautista', 'Antonio Triguero Noriega'] 

Confusion matrix:

[[16  0  0  0]
 [ 0  8  1  0]
 [ 1  6 11  0]
 [ 0  0  0 17]] 

Outcome:

  :) HIT  = 52, (86.67%)
  :( FAIL = 8, (13.33%)
